# פרוייקט בניתוח נתונים מתקדם בפייתון
### מקסים פולק 209383322
### [Github link](https://github.com/Chikapikaa/Advanced-Data-Analysis)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json

In [ ]:
my_url = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17650,17793"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'} # This is chrome, you can set whatever browser you like
api_key = input("Enter Your API Key Here !")
html = requests.get(my_url,headers= headers)
if html.status_code == 200:
    print("Success")
else:
    print("Failure")

In [ ]:
ads_links = []
pageindex = 1
while True:
    url = f"{my_url}&pageindex={pageindex}"
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    main = soup.find('main')
    if not main:
        print("Finish")
        break
    ads = main.find_all("div", class_="card-body p-md-3")
    if not ads:
        break
    for ad in ads:
        a_tag = ad.find('a')
        if a_tag and a_tag.get('href'):
            full_url = "https://www.ad.co.il" + a_tag.get('href')
            ads_links.append(full_url)
    pageindex += 1
print(f"\nסה״כ {len(ads_links)} מודעות נאספו.")
print(ads_links)

In [ ]:
def extract_apartment_data_from_ad_page(url):
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
        html = requests.get(url,headers= headers)
        if html.status_code == 200:
            print("Success")
        else:
            print("Failure")
        try:
            soup = BeautifulSoup(html.content, "html.parser")
            ##### creating dictionary for part of the details ####   
            table = soup.find("table" , class_="table table-sm mb-4")
            details_dict = {}
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                if len(cells) == 2:
                    key = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True)
                    details_dict[key] = value
            ## קומות        
            floors = details_dict['קומה'].split()
            if "קרקע" in floors[0]:
                floor_num = 0
            else: 
                floor_num = int(floors[0])
            if "קרקע" in floors[2]:
                number_of_floors = 0
            else: 
                number_of_floors = int(floors[2])
            ## תאריך כניסה
            entry = details_dict.get('תאריך כניסה', '').strip()

            if entry == "" or entry == "מיידית":
                days_to_enter = 0
            else:
                days_to_enter = entry  
            ###### more info 
            info = soup.find("p" , class_="text-word-break").text
            


            ###### Boolean shit     
            booleans = soup.find("div", class_ = "card-icons flex-wrap d-flex h-100")
            features = {}
            icons = booleans.find_all("div", class_="card-icon")
            for icon in icons:
                label = icon.find("span").get_text(strip=True)
                icon_class = icon.find("i")["class"]
                has_feature = 1 if "fa-check" in icon_class else 0
                features[label] = has_feature


            ##### Price
            price_str = soup.find("div" , class_ = "d-flex justify-content-between").find_all("h2")[1].text
            price = float(price_str.replace("₪", "").replace(",", "").strip())


            ##### Imgs            
            stam = soup.find("div", class_="col-12 d-flex mt-3 justify-content-center flex-wrap")
            Imgs = len(stam.find_all("div", class_="justify-content-center px-1")) if stam else 0
            
            ##### distance-matrix
            
            try:
                origin = details_dict.get("כתובת", "") + ", תל אביב"
                destination = "Dizengoff Square, Tel Aviv"

                dm_url = "https://maps.googleapis.com/maps/api/distancematrix/json"

                params = {
                    "origins": origin,
                    "destinations": destination,
                    "key": api_key,
                    "language": "he",
                    "units": "metric"
                }

                response = requests.get(dm_url, params=params)
                data = response.json()

                if response.status_code == 200:
                    distance_meters = data['rows'][0]['elements'][0]['distance']['value']
                    distance_km = round(distance_meters / 1000, 2)
                    distance_from_center = distance_km
                else:
                    print("שגיאה ב-Distance Matrix:", response.status_code)
                    distance_from_center = np.nan

            except Exception as e:
                print(f"שגיאה בשירות Distance Matrix: {e}")
                distance_from_center = np.nan
            
            ### החזרות
            return {
                "property_type": str(details_dict["פרטי הנכס"]),
                "neighborhood": str(details_dict["שכונה"]),
                "address": str(details_dict['כתובת']),
                "room_num": float(details_dict['חדרים']),
                "floor": int(floor_num), 
                "area": int(details_dict['שטח בנוי']),
                "garden_area": int(details_dict.get('שטח גינה', 0)), 
                "days_to_enter": int(days_to_enter),
                "num_of_payments": int(details_dict['תשלומים בשנה']),
                "monthly_arnona": int(details_dict['ארנונה בחודש']) if 'ארנונה בחודש' in details_dict else np.nan,
                "building_tax": int(details_dict['ועד בית בחודש']) if 'ועד בית בחודש' in details_dict else np.nan,
                "total_floors": int(number_of_floors),
                "description": info.replace("\r\n", " ").strip(),
                "has_parking": int(features['חניה']),
                "has_stotsge": int(features['מחסן']),
                "elevator": int(features['מעלית']),
                "ac": int(features['מזגן']),
                "handicap": int(features['נגישות']),
                "has_bars": int(features['סורגים']),
                "has_safe_room": int(features['ממ"ד']),
                "has_balcon": int(features['מרפסת']),
                "is_furnished": int(features['מרוהטת']),
                "is_renovated": int(features['משופצת']),
                "price": float(price),
                "num_of_images": int(Imgs),
                "distance_from_center": distance_from_center,
            }

        except Exception as e:
            print(f"שגיאה במודעה {url}: {e}")
            return None

In [ ]:
extract_apartment_data_from_ad_page("https://www.ad.co.il/ad/16191410")

In [ ]:
all_ads_data = []
for i, link in enumerate(ads_links, start=1):
    print(f"מעבד מודעה {i} מתוך {len(ads_links)}: {link}")
    data = extract_apartment_data_from_ad_page(link)
    if data:
        all_ads_data.append(data)

In [ ]:
df = pd.DataFrame(all_ads_data)

In [ ]:
df.head(30)

In [ ]:
df.to_csv("tel_aviv_rent_data.csv", index=False, encoding="utf-8-sig")